# Data Locality

## Load Reference Data

In [25]:
import pandas as pd

### Load State & Region Data

In [26]:
df = pd.read_csv('data/constructed-output-data.csv', encoding='utf-8')
df = df.filter(items=['state', 'region', 'latitude', 'longitude']).drop_duplicates()
df

,state,region,latitude,longitude
0,Connecticut,Northeast,41.6219,-72.7273
1,Delaware,Northeast,38.9896,-75.5050
2,Maine,Northeast,45.3695,-69.2428
3,Maryland,Northeast,39.0550,-76.7909
4,Massachusetts,Northeast,42.2596,-71.8083
5,New Hampshire,Northeast,43.6805,-71.5811
6,New Jersey,Northeast,40.1907,-74.6728
7,New York,Northeast,42.9538,-75.5268
8,Pennsylvania,Northeast,40.8781,-77.7996
9,Rhode Island,Northeast,41.6762,-71.5562


### Connect to Redis

In [27]:
import redis
conn = redis.Redis()

### Load State Data

In [31]:
for index, row in df.iterrows():
    state = row['state']
    region = row['region']
    latitude = row['latitude']
    longitude = row['longitude']
    conn.geoadd('woodchuckin:states:coords', longitude, latitude, state)
    conn.set(f'woodchuckin:states:{state}:region', region)
    conn.sadd(f'woodchuckin:regions:{region}:state', state)